In [ ]:
import autograd as ad
from autograd import grad, jacobian, hessian
import autograd.numpy as np

In [ ]:
def func1(x):
    """
    Objective function 1 whose roots are the ones to find.
    """
    return (x[0]+x[1]+x[2]-5)**2 + 3*(x[0]-x[1])**2 + 2*(x[1]-2*x[2])**2

In [ ]:
def func2(x):
    """
    Objective function 2 whose roots are the ones to find.
    """
    return 100*((x[1]-x[0])**2) + (1-x[0])**2

In [ ]:
jacobian_func = jacobian(func1)
hessian_func = hessian(func1)

In [ ]:
def NewtonMethod(func, jacob_f, hessian_f, w0, epsilon = 0.001, nMax = 100):
    #Initializating variables, iter_x, iter_x are used to plot results
    i = 0
    # iter_x, iter_y, iter_count = np.empty(0),np.empty(0),np.empty(0)
    
    jacobian_w = jacob_f(w0)
    hessian_w_inv = np.linalg.inv(hessian_f(w0))
    D = np.dot(hessian_w_inv, jacobian_w)
    error = np.linalg.norm(w0 - (w0 - D))
    #Looping as long as error is greater than epsilon or maximum number of iterations is not reached:
    while error > epsilon and i < nMax:
        i +=1
        # iter_x = np.append(iter_x,x)
        # iter_y = np.append(iter_y,func(x))
        # iter_count = np.append(iter_count ,i)
        w0 = (w0 - D)
        jacobian_w = jacob_f(w0)
        hessian_w_inv = np.linalg.inv(hessian_f(w0))
        D = np.dot(hessian_w_inv, jacobian_w)
        error = np.linalg.norm(w0 - (w0 - D))
        print("Iteration Number : {}".format(i))
        print("Error : {}".format(error))
        print("Minimizer : {}".format(w0))

In [ ]:
W0 = np.array([0, 0, 0], dtype=float)
NewtonMethod(func1, jacobian_func, hessian_func, W0)

Iteration Number : 1
Error : 0.0
Minimizer : [2. 2. 1.]


In [ ]:
W0 = np.array([0, 0.005, -2], dtype=float)
NewtonMethod(func1, jacobian_func, hessian_func, W0)

Iteration Number : 1
Error : 1.1102230246251565e-16
Minimizer : [2. 2. 1.]


In [ ]:
W0 = np.array([200,300,5000], dtype=float)
NewtonMethod(func1, jacobian_func, hessian_func, W0)

Iteration Number : 1
Error : 5.85238816540084e-13
Minimizer : [2. 2. 1.]


In [ ]:
def func2(x):
    """
    Objective function 2 whose roots are the ones to find.
    """
    return 100*((x[1]-x[0]**2)**2) + (1-x[0])**2

In [ ]:
jacobian_func = jacobian(func2)
hessian_func = hessian(func2)

In [ ]:
W0 = np.array([-1.2, 1], dtype=float)
NewtonMethod(func2, jacobian_func, hessian_func, W0)

Iteration Number : 1
Error : 4.95094472322468
Minimizer : [-1.1752809   1.38067416]
Iteration Number : 2
Error : 3.75785864343104
Minimizer : [ 0.76311487 -3.17503385]
Iteration Number : 3
Error : 0.4317760753884038
Minimizer : [0.76342968 0.58282478]
Iteration Number : 4
Error : 0.05596406747375653
Minimizer : [0.99999531 0.94402732]
Iteration Number : 5
Error : 9.62477796681768e-06
Minimizer : [0.9999957  0.99999139]


In [ ]:
W0 = np.array([0, 1/400+10**-12], dtype=float)
NewtonMethod(func2, jacobian_func, hessian_func, W0)

Iteration Number : 1
Error : 3.995006440463911
Minimizer : [2. 0.]
Iteration Number : 2
Error : 4.114254914080283
Minimizer : [1.99875156 3.99500625]
Iteration Number : 3
Error : 0.9968799797602049
Minimizer : [1.00031123 0.00373948]
Iteration Number : 4
Error : 0.0006926388451605293
Minimizer : [1.00030968 1.00061946]


In [ ]:
W0 = np.array([0, 0.005], dtype=float)
NewtonMethod(func2, jacobian_func, hessian_func, W0)

In [ ]:
# Newton method fails for the vector [0, 0.005] --> Singular Matrix which cannot be Inverted
# --> Newton method is not a global optimizer, it cannot guarantee the convergence for every starting point

In [ ]:
def GlobalNewtonMethod(func, jacob_f, hessian_f, w0, epsilon = 0.001, nMax = 100, Mu = 2, c = 0.0001):
    i = 0
    
    jacobian_w = jacob_f(w0)
    min_eigen = -min(np.linalg.eigvals(hessian_f(w0)))
    Yk = Mu * max(10**-10, min_eigen)
    hessian_w_reg = hessian_f(w0) + np.dot(np.identity(hessian_f(w0).shape[0]), Yk)
    hessian_w_reg_inv = np.linalg.inv(hessian_w_reg)
    D = -np.dot(hessian_w_reg_inv, jacobian_w)
    error = np.linalg.norm(w0 - (w0 + D))
    #Looping as long as error is greater than epsilon or maximum number of iterations is not reached:
    while error > epsilon and i < nMax:
        i +=1
        w0 = (w0 + D)
        jacobian_w = jacob_f(w0)
        while func(w0+D) >= func(w0) + c*np.dot(np.transpose(D),jacobian_w):
          Yk = Mu*Yk
          hessian_w_reg = hessian_func(w0) + np.dot(np.identity(hessian_func(w0).shape[0]), Yk)
          hessian_w_reg_inv = np.linalg.inv(hessian_w_reg)
          D = -np.dot(hessian_w_reg_inv, jacobian_w)

        error = np.linalg.norm(w0 - (w0 + D))

    print("Iteration Number : {}".format(i))
    print("Error : {}".format(error))
    print("Minimizer : {}".format(w0))


In [ ]:
W0 = np.array([-1.2, 1], dtype=float)
GlobalNewtonMethod(func=func2, jacob_f=jacobian_func, hessian_f=hessian_func, w0=W0)

Iteration Number : 59
Error : 0.0008464258865713102
Minimizer : [ 0.0425455  -0.01105647]


In [ ]:
W0 = np.array([0, 1/400+10**-12], dtype=float)
GlobalNewtonMethod(func=func2, jacob_f=jacobian_func, hessian_f=hessian_func, w0=W0)

Iteration Number : 79
Error : 0.00034205414867861386
Minimizer : [1.18541131 1.40620071]


In [ ]:
W0 = np.array([0, 0.005], dtype=float)
GlobalNewtonMethod(func=func2, jacob_f=jacobian_func, hessian_f=hessian_func, w0=W0)

Iteration Number : 71
Error : nan
Minimizer : [1.e+10 1.e+20]


In [ ]:
# The Global version of the Newton method converges for the vector [0, 0.005], which was divergent for the previous version of the method.

In [ ]:
for mu in [0, 2, 5, 7, 17, 120]:
  print("Mu : {}".format(mu))
  GlobalNewtonMethod(func=func2, jacob_f=jacobian_func, hessian_f=hessian_func, w0=W0, Mu=mu)
  print("###############################")

Mu : 0
Iteration Number : 1
Error : 3.76822190084106e-15
Minimizer : [1. 1.]
###############################
Mu : 2
Iteration Number : 1
Error : 2.832097600274806e-10
Minimizer : [1. 1.]
###############################
Mu : 5
Iteration Number : 1
Error : 7.079695247253446e-10
Minimizer : [1. 1.]
###############################
Mu : 7
Iteration Number : 1
Error : 9.911834452894007e-10
Minimizer : [1. 1.]
###############################
Mu : 17
Iteration Number : 1
Error : 2.4071301881692756e-09
Minimizer : [1. 1.]
###############################
Mu : 120
Iteration Number : 1
Error : 1.6991680508820098e-08
Minimizer : [0.99999999 0.99999999]
###############################


In [ ]:
import time

In [ ]:
for c in [10**-5, 10**-3, 0.1, 1, 10, 100]:
  print("C : {}".format(c))
  start_time = time.time()
  GlobalNewtonMethod(func=func2, jacob_f=jacobian_func, hessian_f=hessian_func, w0=W0, c=c)
  print("Processing time(s): {}".format(time.time() - start_time))
  print("###############################")

C : 1e-05
Iteration Number : 1
Error : 2.832097600274806e-10
Minimizer : [1. 1.]
Processing time(s): 0.018755197525024414
###############################
C : 0.001
Iteration Number : 1
Error : 2.832097600274806e-10
Minimizer : [1. 1.]
Processing time(s): 0.016400814056396484
###############################
C : 0.1
Iteration Number : 1
Error : 2.832097600274806e-10
Minimizer : [1. 1.]
Processing time(s): 0.015430927276611328
###############################
C : 1
Iteration Number : 1
Error : 1.6054197309469919e-13
Minimizer : [1. 1.]
Processing time(s): 0.22468876838684082
###############################
C : 10
Iteration Number : 1
Error : nan
Minimizer : [1. 1.]
Processing time(s): 5.031253337860107
###############################
C : 100
Iteration Number : 1
Error : nan
Minimizer : [1. 1.]
Processing time(s): 4.960794448852539
###############################


In [ ]:
# The method is not very sensitive to changes for values in Mu or C (converges in one iteration + very small change in the error)
# the only noticeable difference is in the Processing time

In [ ]:
def logistic_loss(X,Y,w,reg_lambda):
    return np.sum(np.log(1 + np.exp(-Y*(X.dot(w))))) + 0.5*reg_lambda*(w**2).sum()

In [ ]:
def logistic_grad(X,Y,w,reg_lambda):
    return (-Y/(1 + np.exp(Y*(X.dot(w)))))*X + reg_lambda*w

In [ ]:
def logistic_hessian(X,Y,w,reg_lambda):
    return (1/(1 + np.exp(Y*(X.dot(w))))) * (np.exp(Y*(X.dot(w)))) * (X.dot(np.transpose(X))) + reg_lambda*np.identity(len(w))

In [ ]:
def subsampled_newton(X,Y,w0,reg_lambda,subsample_size):
    loss = logistic_loss(X,Y,w0,reg_lambda)
    grad = logistic_grad(X,Y,w0,reg_lambda)
    hessian = logistic_hessian(X,Y,w0,reg_lambda)
    n,d = X.shape
    
    max_niters = 150
    error = 100
    epsilon = 0.005
    c = 0.0001
    Mu = 2

    start_time = time.time()
    i = 0
    while error > epsilon and i < max_niters:
        i +=1

        # compute the subsample Hessians
        idx = np.random.choice(n,subsample_size)
        p_sub = 1.0/subsample_size
        X_sub = X[idx,:]

        # define gradient for subsampling problem:
        grad_sub = logistic_grad(X_sub,Y[idx],w0,reg_lambda)
        grad_sub = grad_sub * p_sub
        
        # define hessian for subsampling problem:
        hess_sub = logistic_hessian(X_sub,Y[idx],w0,reg_lambda)
        hess_sub = hess_sub * p_sub

        min_eigen = -min(np.linalg.eigvals(hess_sub))
        Yk = Mu * max(10**-10, min_eigen)
        hessian_w_reg = hess_sub + np.dot(np.identity(subsample_size), Yk)
        hessian_w_reg_inv = np.linalg.inv(hessian_w_reg)
        D = -np.dot(hessian_w_reg_inv, grad_sub)
        error = np.linalg.norm(w0 - (w0 + D))

        while logistic_loss(w0+D) >= logistic_loss(w0) + c*np.dot(np.transpose(D),grad_sub):
          Yk = Mu*Yk
          hessian_w_reg = hess_sub + np.dot(np.identity(subsample_size), Yk)
          hessian_w_reg_inv = np.linalg.inv(hessian_w_reg)
          D = -np.dot(hessian_w_reg_inv, grad_sub)

        error = np.linalg.norm(w0 - (w0 + D))
        w0 = (w0 + D)
        print("Iteration Number : {}".format(i))
        print("Error : {}".format(error))
        print("Minimizer : {}".format(w0))

